In [ ]:
import os
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

# Wenn dein Notebook in notebooks/ liegt und .env im Projekt-Root:
load_dotenv(Path("..") / ".env")

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")          # <- dein Key
DB_PASS = os.getenv("DB_PASSWORD")

assert DB_HOST and DB_NAME and DB_USER, "Fehlende DB-Variablen (DB_HOST/DB_NAME/DB_USER)"

engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# Test
with engine.connect() as conn:
    print(conn.execute(text("SELECT current_user, current_database()")).fetchone())


('postgres', 'mimic')


In [5]:
import pandas as pd
from sqlalchemy import text

AKI_ONLY = True  # False = alle Patienten/Aufenthalte

def q(sql: str) -> pd.DataFrame:
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn)


In [6]:
sql_icd_procedures = f"""
WITH aki_hadm AS (
  SELECT DISTINCT hadm_id
  FROM diagnoses_icd
  WHERE icd9_code LIKE '584%'
)
SELECT
  p.icd9_code,
  d.long_title AS procedure_name,
  COUNT(*) AS n
FROM procedures_icd p
LEFT JOIN d_icd_procedures d
  ON d.icd9_code = p.icd9_code
{"JOIN aki_hadm a ON a.hadm_id = p.hadm_id" if AKI_ONLY else ""}
GROUP BY p.icd9_code, d.long_title
ORDER BY n DESC;
"""
df_icd_proc = q(sql_icd_procedures)
df_icd_proc.head(30)


,icd9_code,procedure_name,n
0,3893,"Venous catheterization, not elsewhere classified",5489
1,9604,Insertion of endotracheal tube,3134
2,966,Enteral infusion of concentrated nutritional s...,2685
3,9672,Continuous invasive mechanical ventilation for...,2412
4,9671,Continuous invasive mechanical ventilation for...,2135
5,9904,Transfusion of packed cells,2075
6,3891,Arterial catheterization,1723
7,3995,Hemodialysis,1333
8,5491,Percutaneous abdominal drainage,1309
9,3324,Closed [endoscopic] biopsy of bronchus,1254


In [7]:
sql_icu_procedures_mv = f"""
WITH aki_hadm AS (
  SELECT DISTINCT hadm_id
  FROM diagnoses_icd
  WHERE icd9_code LIKE '584%'
)
SELECT
  pe.itemid,
  di.label,
  COUNT(*) AS n
FROM procedureevents_mv pe
JOIN icustays i
  ON i.icustay_id = pe.icustay_id
LEFT JOIN d_items di
  ON di.itemid = pe.itemid
{"JOIN aki_hadm a ON a.hadm_id = i.hadm_id" if AKI_ONLY else ""}
GROUP BY pe.itemid, di.label
ORDER BY n DESC;
"""
df_icu_proc = q(sql_icu_procedures_mv)
df_icu_proc.head(30)


,itemid,label,n
0,225459,Chest X-Ray,12922
1,224275,20 Gauge,8378
2,224277,18 Gauge,5595
3,225402,EKG,5256
4,225401,Blood Cultured,4986
5,225752,Arterial Line,3944
6,224263,Multi Lumen,3343
7,225792,Invasive Ventilation,3325
8,225454,Urine Culture,2587
9,221214,CT scan,2516


In [8]:
sql_icu_inputs_mv = f"""
WITH aki_hadm AS (
  SELECT DISTINCT hadm_id
  FROM diagnoses_icd
  WHERE icd9_code LIKE '584%'
)
SELECT
  ie.itemid,
  di.label,
  COUNT(*) AS n_events
FROM inputevents_mv ie
JOIN icustays i
  ON i.icustay_id = ie.icustay_id
LEFT JOIN d_items di
  ON di.itemid = ie.itemid
{"JOIN aki_hadm a ON a.hadm_id = i.hadm_id" if AKI_ONLY else ""}
GROUP BY ie.itemid, di.label
ORDER BY n_events DESC;
"""
df_inputs = q(sql_icu_inputs_mv)
df_inputs.head(30)


,itemid,label,n_events
0,225158,NaCl 0.9%,257403
1,220949,Dextrose 5%,197052
2,225943,Solution,95844
3,222168,Propofol,59385
4,221906,Norepinephrine,59022
5,223258,Insulin - Regular,47357
6,225799,Gastric Meds,45710
7,226452,PO Intake,44819
8,221744,Fentanyl,41046
9,221668,Midazolam (Versed),39198


In [12]:
def contains(df: pd.DataFrame, col: str, term: str, n=30):
    mask = df[col].fillna("").str.lower().str.contains(term.lower())
    return df.loc[mask].head(n)

# Beispiele (anpassen wie du willst)
print(contains(df_inputs, "label", "norepi", 50), contains(df_inputs, "label", "vasopress", 50) )   # Vasopressin
print(contains(df_inputs, "label", "epine", 50) )       # Adrenalin
print(contains(df_inputs, "label", "furosem", 50) )     # Furosemid
print(contains(df_icu_proc, "label", "dial", 50) )      # Dialyse (falls so gelabelt)
print(contains(df_icd_proc, "procedure_name", "dial", 50) )


   itemid           label  n_events
4  221906  Norepinephrine     59022     itemid        label  n_events
63  222315  Vasopressin      3711
    itemid           label  n_events
4   221906  Norepinephrine     59022
70  221289     Epinephrine      2894
    itemid                       label  n_events
15  221794          Furosemide (Lasix)     26010
87  228340  Furosemide (Lasix) 500/100      1670
     itemid                      label     n
16   225802            Dialysis - CRRT  1689
23   225441               Hemodialysis  1070
24   224270          Dialysis Catheter  1050
107  227714  Pericardial Drain Removed     5
113  225805        Peritoneal Dialysis     2
    icd9_code                                     procedure_name     n
7        3995                                       Hemodialysis  1333
10       3895          Venous catheterization for renal dialysis  1239
271      5498                                Peritoneal dialysis    19
293      3927                Arteriovenostomy fo

In [13]:
sql_inputs_unique_stays = f"""
WITH aki_hadm AS (
  SELECT DISTINCT hadm_id
  FROM diagnoses_icd
  WHERE icd9_code LIKE '584%'
)
SELECT
  di.label,
  COUNT(DISTINCT ie.icustay_id) AS n_icustays_with_item
FROM inputevents_mv ie
JOIN icustays i ON i.icustay_id = ie.icustay_id
LEFT JOIN d_items di ON di.itemid = ie.itemid
{"JOIN aki_hadm a ON a.hadm_id = i.hadm_id" if AKI_ONLY else ""}
GROUP BY di.label
ORDER BY n_icustays_with_item DESC;
"""
df_inputs_stays = q(sql_inputs_unique_stays)
df_inputs_stays.head(30)


,label,n_icustays_with_item
0,NaCl 0.9%,6123
1,Dextrose 5%,5594
2,PO Intake,4714
3,Solution,3544
4,Heparin Sodium (Prophylaxis),3496
5,Vancomycin,3435
6,Magnesium Sulfate,3113
7,Potassium Chloride,2789
8,Furosemide (Lasix),2743
9,Pantoprazole (Protonix),2503


In [14]:
sql_intervention_categories = f"""
WITH aki_hadm AS (
  SELECT DISTINCT hadm_id
  FROM diagnoses_icd
  WHERE icd9_code LIKE '584%'
),
events AS (
  SELECT
    ie.icustay_id,
    LOWER(di.label) AS label
  FROM inputevents_mv ie
  JOIN icustays i ON i.icustay_id = ie.icustay_id
  LEFT JOIN d_items di ON di.itemid = ie.itemid
  {"JOIN aki_hadm a ON a.hadm_id = i.hadm_id" if AKI_ONLY else ""}
),
classified AS (
  SELECT
    icustay_id,
    CASE
      WHEN label LIKE '%norepi%' OR label LIKE '%noradrenal%' OR
           label LIKE '%vasopress%' OR label LIKE '%epine%' OR
           label LIKE '%adrenal%' OR label LIKE '%dopamine%'
        THEN 'vasopressors'

      WHEN label LIKE '%furosem%' OR label LIKE '%bumetanide%' OR
           label LIKE '%torsemide%'
        THEN 'diuretics'

      WHEN label LIKE '%saline%' OR label LIKE '%ringer%' OR
           label LIKE '%plasma-lyte%'
        THEN 'fluids'

      WHEN label LIKE '%albumin%'
        THEN 'albumin'

      ELSE NULL
    END AS category
  FROM events
)
SELECT
  category,
  COUNT(DISTINCT icustay_id) AS n_icustays
FROM classified
WHERE category IS NOT NULL
GROUP BY category
ORDER BY n_icustays DESC;
"""
df_categories = q(sql_intervention_categories)
df_categories


,category,n_icustays
0,diuretics,2760
1,vasopressors,2042
2,albumin,1174
3,fluids,48


In [16]:
sql_aki_intervention_overview = """
WITH aki_hadm AS (
  SELECT DISTINCT hadm_id
  FROM diagnoses_icd
  WHERE icd9_code LIKE '584%'
),

-- ICU input events (meds / fluids actually given)
inputs AS (
  SELECT
    ie.icustay_id,
    LOWER(di.label) AS label
  FROM inputevents_mv ie
  JOIN icustays i ON i.icustay_id = ie.icustay_id
  JOIN aki_hadm a ON a.hadm_id = i.hadm_id
  LEFT JOIN d_items di ON di.itemid = ie.itemid
),

-- ICU procedures (e.g. ventilation, dialysis)
procedures AS (
  SELECT
    pe.icustay_id,
    LOWER(di.label) AS label
  FROM procedureevents_mv pe
  JOIN icustays i ON i.icustay_id = pe.icustay_id
  JOIN aki_hadm a ON a.hadm_id = i.hadm_id
  LEFT JOIN d_items di ON di.itemid = pe.itemid
),

-- ICD procedures (hospital coding, dialysis backup)
icd_procs AS (
  SELECT
    i.icustay_id,
    LOWER(dp.long_title) AS label
  FROM procedures_icd p
  JOIN d_icd_procedures dp ON dp.icd9_code = p.icd9_code
  JOIN icustays i ON i.hadm_id = p.hadm_id
  JOIN aki_hadm a ON a.hadm_id = p.hadm_id
),

all_events AS (
  SELECT icustay_id, label FROM inputs
  UNION ALL
  SELECT icustay_id, label FROM procedures
  UNION ALL
  SELECT icustay_id, label FROM icd_procs
),

classified AS (
  SELECT
    icustay_id,
    CASE
      WHEN label LIKE '%norepi%' OR label LIKE '%noradrenal%'
        OR label LIKE '%vasopress%'
        OR label LIKE '%epine%' OR label LIKE '%adrenal%'
        OR label LIKE '%dopamine%'
        THEN 'vasopressors'

      WHEN label LIKE '%furosem%' OR label LIKE '%bumetanide%'
        OR label LIKE '%torsemide%'
        THEN 'diuretics'

      WHEN label LIKE '%saline%'
        OR label LIKE '%nacl%'
        OR label LIKE '%ringer%'
        OR label LIKE '%lactated%'
        OR label LIKE '%lr%'
        OR label LIKE '%plasma%'
        OR label LIKE '%d5w%'
        OR label LIKE '%iv fluid%'
        THEN 'fluids'


      WHEN label LIKE '%albumin%'
        THEN 'albumin'

      WHEN label LIKE '%dial%' OR label LIKE '%cvvh%'
        OR label LIKE '%hemodial%'
        THEN 'dialysis_rrt'

      WHEN label LIKE '%vent%' OR label LIKE '%intubat%'
        OR label LIKE '%respirat%'
        THEN 'mechanical_ventilation'

      WHEN label LIKE '%packed red%' OR label LIKE '%platelet%'
        OR label LIKE '%plasma%'
        THEN 'blood_products'

      ELSE NULL
    END AS intervention
  FROM all_events
)

SELECT
  intervention,
  COUNT(DISTINCT icustay_id) AS n_icustays
FROM classified
WHERE intervention IS NOT NULL
GROUP BY intervention
ORDER BY n_icustays DESC;
"""

df_aki_interventions = q(sql_aki_intervention_overview)
df_aki_interventions


,intervention,n_icustays
0,fluids,6310
1,mechanical_ventilation,6254
2,blood_products,3166
3,diuretics,2760
4,vasopressors,2552
5,dialysis_rrt,1832
6,albumin,1174


In [18]:
sql_aki_intervention_overview_final = """
WITH aki_hadm AS (
  SELECT DISTINCT hadm_id
  FROM diagnoses_icd
  WHERE icd9_code LIKE '584%'
),

inputs AS (
  SELECT
    ie.icustay_id,
    LOWER(COALESCE(di.label,'')) AS label
  FROM inputevents_mv ie
  JOIN icustays i ON i.icustay_id = ie.icustay_id
  JOIN aki_hadm a ON a.hadm_id = i.hadm_id
  LEFT JOIN d_items di ON di.itemid = ie.itemid
),

procedures AS (
  SELECT
    pe.icustay_id,
    LOWER(COALESCE(di.label,'')) AS label
  FROM procedureevents_mv pe
  JOIN icustays i ON i.icustay_id = pe.icustay_id
  JOIN aki_hadm a ON a.hadm_id = i.hadm_id
  LEFT JOIN d_items di ON di.itemid = pe.itemid
),

icd_procs AS (
  SELECT
    i.icustay_id,
    LOWER(COALESCE(dp.long_title,'')) AS label
  FROM procedures_icd p
  JOIN d_icd_procedures dp ON dp.icd9_code = p.icd9_code
  JOIN icustays i ON i.hadm_id = p.hadm_id
  JOIN aki_hadm a ON a.hadm_id = p.hadm_id
),

all_events AS (
  SELECT icustay_id, label FROM inputs
  UNION ALL
  SELECT icustay_id, label FROM procedures
  UNION ALL
  SELECT icustay_id, label FROM icd_procs
),

classified AS (
  SELECT
    icustay_id,
    CASE
      -- Fluids (clean)
      WHEN (
        label LIKE '%nacl%' OR label LIKE '%normal saline%' OR label = 'ns'
        OR label LIKE '%0.9%' OR label LIKE '%0.45%' OR label LIKE '%1/2ns%'
        OR label LIKE '%d5 1/2ns%' OR label LIKE '%half ns%'
        OR label = 'lr' OR label LIKE '%lactated%' OR label LIKE '%ringer%'
        OR label LIKE '%dextrose%' OR label LIKE '%d5w%' OR label LIKE '%d10w%'
        OR label LIKE '%dextrose 5%' OR label LIKE '%dextrose 10%' OR label LIKE '%dextrose 50%'
        OR label LIKE '%sterile water%' OR label LIKE '%free water%'
        OR label LIKE '%plasma-lyte%' OR label LIKE '%plasmalyte%'
      )
      AND label NOT LIKE '%po intake%'
      AND label NOT LIKE '%pre-admission intake%'
      AND label NOT LIKE '%solution%'
      AND label NOT LIKE '%piggyback%'
      THEN 'fluids'

      -- Vasopressors (complete)
      WHEN label LIKE '%norepi%' OR label LIKE '%noradren%' OR label LIKE '%levophed%' OR label = 'ne'
        OR label LIKE '%vasopress%'
        OR label LIKE '%epine%' OR label LIKE '%adrenalin%'
        OR label LIKE '%phenyleph%'
        OR label LIKE '%dopamine%'
        OR label LIKE '%dobutamine%'
      THEN 'vasopressors'

      -- Diuretics
      WHEN label LIKE '%furosem%' OR label LIKE '%lasix%'
        OR label LIKE '%bumetanide%' OR label LIKE '%torsemide%'
      THEN 'diuretics'

      -- Albumin
      WHEN label LIKE '%albumin%'
      THEN 'albumin'

      -- Blood products
      WHEN label LIKE '%packed red blood%' OR label LIKE '%prbc%'
        OR label LIKE '%fresh frozen plasma%' OR label LIKE '%ffp%'
        OR label LIKE '%platelet%'
      THEN 'blood_products'

      -- Dialysis / RRT
      WHEN label LIKE '%dial%' OR label LIKE '%cvvh%' OR label LIKE '%cvvhd%' OR label LIKE '%cvvhdf%'
        OR label LIKE '%hemodial%' OR label LIKE '%haemodial%'
      THEN 'dialysis_rrt'

      -- Mechanical ventilation
      WHEN label LIKE '%vent%' OR label LIKE '%intubat%' OR label LIKE '%respirat%'
      THEN 'mechanical_ventilation'

      ELSE NULL
    END AS intervention
  FROM all_events
)

SELECT
  intervention,
  COUNT(DISTINCT icustay_id) AS n_icustays
FROM classified
WHERE intervention IS NOT NULL
GROUP BY intervention
ORDER BY n_icustays DESC;
"""
df_overview = q(sql_aki_intervention_overview_final)
df_overview


,intervention,n_icustays
0,fluids,6438
1,mechanical_ventilation,6254
2,blood_products,3376
3,vasopressors,3039
4,diuretics,2760
5,dialysis_rrt,1832
6,albumin,1174


In [17]:
sql_top_input_labels = """
WITH aki_hadm AS (
  SELECT DISTINCT hadm_id
  FROM diagnoses_icd
  WHERE icd9_code LIKE '584%'
)
SELECT
  LOWER(di.label) AS label,
  COUNT(DISTINCT ie.icustay_id) AS n_icustays
FROM inputevents_mv ie
JOIN icustays i ON i.icustay_id = ie.icustay_id
JOIN aki_hadm a ON a.hadm_id = i.hadm_id
LEFT JOIN d_items di ON di.itemid = ie.itemid
GROUP BY LOWER(di.label)
ORDER BY n_icustays DESC
LIMIT 200;
"""
df_top_labels = q(sql_top_input_labels)
df_top_labels.head(50)


,label,n_icustays
0,nacl 0.9%,6123
1,dextrose 5%,5594
2,po intake,4714
3,solution,3544
4,heparin sodium (prophylaxis),3496
5,vancomycin,3435
6,magnesium sulfate,3113
7,potassium chloride,2789
8,furosemide (lasix),2743
9,pantoprazole (protonix),2503


In [19]:
df_creat_items = q("""
SELECT itemid, label
FROM d_labitems
WHERE LOWER(label) LIKE '%creatinine%'
ORDER BY itemid;
""")
df_creat_items.head(10)


,itemid,label
0,50841,"Creatinine, Ascites"
1,50912,Creatinine
2,51021,"Creatinine, Joint Fluid"
3,51032,"Creatinine, Body Fluid"
4,51052,"Creatinine, Pleural"
5,51067,24 hr Creatinine
6,51070,"Albumin/Creatinine, Urine"
7,51073,"Amylase/Creatinine Ratio, Urine"
8,51080,Creatinine Clearance
9,51081,"Creatinine, Serum"


In [20]:
from sqlalchemy import text

CREAT_ITEMID = 50912

sql_create_aki_stages_view = f"""
CREATE OR REPLACE VIEW aki_stages AS
WITH icu AS (
  SELECT subject_id, hadm_id, icustay_id, intime, outtime
  FROM icustays
),
creat AS (
  SELECT
    hadm_id,
    charttime,
    valuenum AS creat
  FROM labevents
  WHERE itemid = {CREAT_ITEMID}
    AND valuenum IS NOT NULL
),
baseline AS (
  SELECT
    i.icustay_id,
    MIN(c.creat) AS baseline_creat
  FROM icu i
  JOIN creat c
    ON c.hadm_id = i.hadm_id
   AND c.charttime BETWEEN i.intime - interval '24 hour' AND i.intime + interval '24 hour'
  GROUP BY i.icustay_id
),
peak AS (
  SELECT
    i.icustay_id,
    MAX(c.creat) AS peak_creat
  FROM icu i
  JOIN creat c
    ON c.hadm_id = i.hadm_id
   AND c.charttime BETWEEN i.intime AND i.intime + interval '7 day'
  GROUP BY i.icustay_id
)
SELECT
  b.icustay_id,
  b.baseline_creat,
  p.peak_creat,
  CASE
    WHEN b.baseline_creat IS NULL OR p.peak_creat IS NULL THEN NULL
    WHEN p.peak_creat >= 4.0 OR p.peak_creat >= 3.0 * b.baseline_creat THEN 3
    WHEN p.peak_creat >= 2.0 * b.baseline_creat THEN 2
    WHEN p.peak_creat >= 1.5 * b.baseline_creat OR (p.peak_creat - b.baseline_creat) >= 0.3 THEN 1
    ELSE 0
  END AS aki_stage
FROM baseline b
JOIN peak p USING (icustay_id);
"""

with engine.begin() as conn:
    conn.execute(text(sql_create_aki_stages_view))

print("Created view: aki_stages")


Created view: aki_stages


In [21]:
df_stage_counts = q("""
SELECT aki_stage, COUNT(*) AS n_icustays
FROM aki_stages
GROUP BY aki_stage
ORDER BY aki_stage;
""")
df_stage_counts


,aki_stage,n_icustays
0,0,33262
1,1,12002
2,2,2150
3,3,4909


In [22]:
sql_stage_x_intervention = """
WITH aki_hadm AS (
  SELECT DISTINCT hadm_id
  FROM diagnoses_icd
  WHERE icd9_code LIKE '584%'
),
inputs AS (
  SELECT
    ie.icustay_id,
    LOWER(COALESCE(di.label,'')) AS label
  FROM inputevents_mv ie
  JOIN icustays i ON i.icustay_id = ie.icustay_id
  JOIN aki_hadm a ON a.hadm_id = i.hadm_id
  LEFT JOIN d_items di ON di.itemid = ie.itemid
),
procedures AS (
  SELECT
    pe.icustay_id,
    LOWER(COALESCE(di.label,'')) AS label
  FROM procedureevents_mv pe
  JOIN icustays i ON i.icustay_id = pe.icustay_id
  JOIN aki_hadm a ON a.hadm_id = i.hadm_id
  LEFT JOIN d_items di ON di.itemid = pe.itemid
),
icd_procs AS (
  SELECT
    i.icustay_id,
    LOWER(COALESCE(dp.long_title,'')) AS label
  FROM procedures_icd p
  JOIN d_icd_procedures dp ON dp.icd9_code = p.icd9_code
  JOIN icustays i ON i.hadm_id = p.hadm_id
  JOIN aki_hadm a ON a.hadm_id = p.hadm_id
),
all_events AS (
  SELECT icustay_id, label FROM inputs
  UNION ALL
  SELECT icustay_id, label FROM procedures
  UNION ALL
  SELECT icustay_id, label FROM icd_procs
),
classified AS (
  SELECT
    icustay_id,
    CASE
      WHEN (
        label LIKE '%nacl%' OR label LIKE '%normal saline%' OR label = 'ns'
        OR label LIKE '%0.9%' OR label LIKE '%0.45%' OR label LIKE '%1/2ns%'
        OR label LIKE '%d5 1/2ns%' OR label LIKE '%half ns%'
        OR label = 'lr' OR label LIKE '%lactated%' OR label LIKE '%ringer%'
        OR label LIKE '%dextrose%' OR label LIKE '%d5w%' OR label LIKE '%d10w%'
        OR label LIKE '%dextrose 5%' OR label LIKE '%dextrose 10%' OR label LIKE '%dextrose 50%'
        OR label LIKE '%sterile water%' OR label LIKE '%free water%'
        OR label LIKE '%plasma-lyte%' OR label LIKE '%plasmalyte%'
      )
      AND label NOT LIKE '%po intake%'
      AND label NOT LIKE '%pre-admission intake%'
      AND label NOT LIKE '%solution%'
      AND label NOT LIKE '%piggyback%'
      THEN 'fluids'

      WHEN label LIKE '%norepi%' OR label LIKE '%noradren%' OR label LIKE '%levophed%' OR label = 'ne'
        OR label LIKE '%vasopress%'
        OR label LIKE '%epine%' OR label LIKE '%adrenalin%'
        OR label LIKE '%phenyleph%'
        OR label LIKE '%dopamine%'
        OR label LIKE '%dobutamine%'
      THEN 'vasopressors'

      WHEN label LIKE '%furosem%' OR label LIKE '%lasix%'
        OR label LIKE '%bumetanide%' OR label LIKE '%torsemide%'
      THEN 'diuretics'

      WHEN label LIKE '%albumin%'
      THEN 'albumin'

      WHEN label LIKE '%packed red blood%' OR label LIKE '%prbc%'
        OR label LIKE '%fresh frozen plasma%' OR label LIKE '%ffp%'
        OR label LIKE '%platelet%'
      THEN 'blood_products'

      WHEN label LIKE '%dial%' OR label LIKE '%cvvh%' OR label LIKE '%cvvhd%' OR label LIKE '%cvvhdf%'
        OR label LIKE '%hemodial%' OR label LIKE '%haemodial%'
      THEN 'dialysis_rrt'

      WHEN label LIKE '%vent%' OR label LIKE '%intubat%' OR label LIKE '%respirat%'
      THEN 'mechanical_ventilation'

      ELSE NULL
    END AS intervention
  FROM all_events
),
dedup AS (
  SELECT DISTINCT icustay_id, intervention
  FROM classified
  WHERE intervention IS NOT NULL
)
SELECT
  s.aki_stage,
  d.intervention,
  COUNT(*) AS n_icustays
FROM dedup d
JOIN aki_stages s
  ON s.icustay_id = d.icustay_id
GROUP BY s.aki_stage, d.intervention
ORDER BY s.aki_stage, n_icustays DESC;
"""
df_stage_interv = q(sql_stage_x_intervention)
df_stage_interv


,aki_stage,intervention,n_icustays
0,0,fluids,2425
1,0,mechanical_ventilation,1837
2,0,blood_products,960
3,0,vasopressors,828
4,0,diuretics,818
5,0,albumin,226
6,0,dialysis_rrt,156
7,1,mechanical_ventilation,2324
8,1,fluids,2307
9,1,blood_products,1222


In [23]:
# Prozent pro Stage in pandas
totals = df_stage_interv.groupby("aki_stage")["n_icustays"].sum().rename("stage_total")
df_stage_interv2 = df_stage_interv.merge(totals, on="aki_stage")
df_stage_interv2["pct_of_stage"] = 100 * df_stage_interv2["n_icustays"] / df_stage_interv2["stage_total"]
df_stage_interv2.sort_values(["aki_stage", "pct_of_stage"], ascending=[True, False]).head(30)


,aki_stage,intervention,n_icustays,stage_total,pct_of_stage
0,0,fluids,2425,7250,33.448276
1,0,mechanical_ventilation,1837,7250,25.337931
2,0,blood_products,960,7250,13.241379
3,0,vasopressors,828,7250,11.420690
4,0,diuretics,818,7250,11.282759
5,0,albumin,226,7250,3.117241
6,0,dialysis_rrt,156,7250,2.151724
7,1,mechanical_ventilation,2324,8981,25.876851
8,1,fluids,2307,8981,25.687563
9,1,blood_products,1222,8981,13.606503
